In [1]:
from pykeen.pipeline import pipeline
from pykeen.triples import TriplesFactory
import pandas as pd
import torch
import numpy as np

In [3]:
# Nota sobre la seguridad cargando modelos serializados
model = torch.load('models/inmuebles_grupo_01.v2/trained_model.pkl', weights_only=False)

RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.

In [4]:
triples_file = 'dataset/dataset_train.tsv.gz'
triples_factory = TriplesFactory.from_path(triples_file,create_inverse_triples=True)
triples_factory

TriplesFactory(num_entities=418172, num_relations=48, create_inverse_triples=True, num_triples=2008458, path="/home/emanuel/Maestria en IS/TopicosIngSoftware/dataset/dataset_train.tsv.gz")

In [5]:
triples_factory.relation_to_id['http://www.w3.org/2002/07/owl#sameAs']

5

In [6]:
df = pd.read_csv(triples_file, sep='\t', header=None, names=['head', 'relation', 'tail'])
heads = df[list(map(lambda x: True if ('pronto.owl#space_site' in x) and (len(x.split('#')[1].split('_')) == 3) else False, df['head'].values))]['head'].values
relations = ['http://www.w3.org/2002/07/owl#sameAs'] * len(heads)
# heads = df['head'].tolist()
# relations = df['relation'].tolist()

heads_idx = [triples_factory.entity_to_id[head] for head in heads]
relations_idx = [triples_factory.relation_to_id[relation] for relation in relations]

In [7]:
hr_batch = torch.tensor(list(zip(heads_idx, relations_idx)))
hr_batch.shape

torch.Size([1420663, 2])

In [8]:
sample = hr_batch[:20]
sample

tensor([[334785,      5],
        [252582,      5],
        [301978,      5],
        [304516,      5],
        [327025,      5],
        [280052,      5],
        [309811,      5],
        [294461,      5],
        [309906,      5],
        [320615,      5],
        [320677,      5],
        [296972,      5],
        [325533,      5],
        [350690,      5],
        [295861,      5],
        [344774,      5],
        [416870,      5],
        [415431,      5],
        [313221,      5],
        [366731,      5]])

In [9]:
torch.cuda.empty_cache()

In [17]:
scores = model.score_t(sample)
scores

tensor([[ -5.4302,  -5.0816,  -6.7288,  ...,  -6.6870,  -8.9306,  -8.5871],
        [ -8.9738,  -8.6056, -10.2839,  ..., -10.2634, -12.5309, -12.1704],
        [ -6.8197,  -6.4671,  -8.1440,  ...,  -8.0994, -10.3663, -10.0077],
        ...,
        [ -3.6246,  -3.3023,  -4.8715,  ...,  -4.8422,  -7.0516,  -6.7066],
        [ -5.8866,  -5.5371,  -7.1964,  ...,  -7.1576,  -9.4119,  -9.0574],
        [ -8.9070,  -8.5412, -10.2245,  ..., -10.2001, -12.4749, -12.1164]],
       device='cuda:1', grad_fn=<NegBackward0>)

In [20]:
scores[2].min()

tensor(-15.8717, device='cuda:1', grad_fn=<MinBackward1>)

In [21]:
scores = scores[1].to('cpu').detach().numpy()

In [65]:
T = 0.208 # Ajusta según la dispersión de los scores
probabilities = np.exp(-scores / T) / np.sum(np.exp(-scores / T))
#probabilities = np.exp(-scores) / np.sum(np.exp(-scores))

In [66]:
probabilities.max()

np.float32(0.14154623)

In [68]:
[(triples_factory.entity_id_to_label[int(l)],probabilities[int(l)]) for l in hr_batch[probabilities[hr_batch[:,0]]>0.005][:,0]]

[]

In [30]:
def get_prob(s):
    return 1 / (1 + np.exp(-s))

In [40]:
get_prob(-1.8)

np.float64(0.14185106490048777)